In [1]:
import cv2 as cv
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

2025-08-07 18:46:44.550790: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-07 18:46:44.858582: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-07 18:46:45.917423: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# import unet
# model = unet.createModel(256,256)
# model.load_weights("model_unet_after_i_style.weights.h5")
# model.summary()

In [3]:
import unet2
model = unet2.build_u2net_lite((256, 256, 3), 2)
model.load_weights("unet2_2.weights.h5")

2025-08-07 18:46:48.899014: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-07 18:46:49.127244: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2025-08-07 18:46:49.127957: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
# model.summary()

In [5]:
def show_image(images: np.array, masks: np.array):
    """Displays images and their corresponding masks.

    Args:
        images (np.array): Array of images.
        masks (np.array): Array of masks.
    """
    # Masks_NORM = mpl.colors.Normalize(vmin=0, vmax=59)
    fig, axes = plt.subplots(images.shape[0], 2, figsize=(10, 10))

    # Handle single image case
    if images.shape[0] == 1:
        axes = [axes]  # Wrap axes in a list if only one image

    for ax, img, msk in zip(axes, images, masks):
        im1 = ax[0].imshow(img, cmap="gray")
        # im2 = ax[1].imshow(msk, cmap='gray', norm=Masks_NORM)
        im2 = ax[1].imshow(msk, cmap='gray')

        ax[0].set_title("Image")
        ax[1].set_title("Mask")

        fig.colorbar(im2, ax=ax[1], orientation='vertical')

    fig.tight_layout()

In [6]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

In [12]:
TEST_IMAGES_PATH = "/home/cat/projects/CV-stuff/latl/images"
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256

import glob
import os

def test_images(model, images_path: str):
    images_paths = glob.glob(os.path.join(images_path, "**", "*.jpg"), recursive=True)
    print(images_paths)
    images = []
    masks = []
    for image_path in images_paths:
        image = cv.imread(image_path, cv.IMREAD_COLOR)
        w,h,ch = image.shape
        image = cv.cvtColor(image, cv.COLOR_BGRA2RGB)
        images.append(image)
        image = cv.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT), interpolation=cv.INTER_NEAREST)
        print(image.shape)
        image = np.expand_dims(image, axis=0)
        image = image.astype(np.float32) / 255.0
        print(image.shape)
        pred_mask = model.predict(image)
        # print("mask shape",pred_mask.shape)
        pred_mask = create_mask(pred_mask)
        # print("create_mask shape",pred_mask.shape)
        # pred_mask = np.squeeze(pred_mask)
        pred_mask = np.array(pred_mask)
        # print("squeeze shape",pred_mask.shape)
        pred_mask = cv.resize(pred_mask, (h, w), interpolation=cv.INTER_NEAREST)
        # print("resize shape",pred_mask.shape)
        masks.append(pred_mask)
        # cv.imwrite("predictions/" + os.path.basename(image_path), pred_mask)
    show_image(np.array(images)[:3], masks[:3])
    show_image(np.array(images)[3:6], masks[3:6])
    show_image(np.array(images)[6:9], masks[6:9])



In [ ]:
test_images(model, TEST_IMAGES_PATH)

['/home/cat/projects/CV-stuff/latl/images/black/PXL_20250409_101951593.jpg', '/home/cat/projects/CV-stuff/latl/images/black/PXL_20250409_102319434.jpg', '/home/cat/projects/CV-stuff/latl/images/black/PXL_20250409_102306293.jpg', '/home/cat/projects/CV-stuff/latl/images/white/PXL_20250409_102225980.jpg', '/home/cat/projects/CV-stuff/latl/images/white/PXL_20250409_102121490.jpg', '/home/cat/projects/CV-stuff/latl/images/white/PXL_20250409_102136903.jpg', '/home/cat/projects/CV-stuff/latl/images/grey/PXL_20250409_102642919.jpg', '/home/cat/projects/CV-stuff/latl/images/grey/PXL_20250409_102601865.jpg', '/home/cat/projects/CV-stuff/latl/images/grey/PXL_20250409_102619587.jpg', '/home/cat/projects/CV-stuff/latl/images/green/PXL_20250409_102436529.jpg', '/home/cat/projects/CV-stuff/latl/images/green/PXL_20250409_102458390.jpg', '/home/cat/projects/CV-stuff/latl/images/green/PXL_20250409_102424514.jpg']
(256, 256, 3)
(1, 256, 256, 3)


I0000 00:00:1754585326.264180  272003 service.cc:145] XLA service 0x782d68015fd0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1754585326.264410  272003 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2050, Compute Capability 8.6
2025-08-07 18:48:46.354871: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-08-07 18:48:46.980045: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
